In [19]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

load_dotenv()
from openai import OpenAI

In [20]:
#Initialize OpenAI 
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai.api_key)

In [21]:
# Ensure CELL INDEX 1 is executed before running this cell
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Hello, how are you doing?"}
    ],
    temperature=0.7,
    max_tokens=100
)
# Print the response
chat_completion.choices[0].message.content

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?"

In [22]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    


In [1]:
prompt = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""".strip()

In [2]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [25]:
print("Welcome to the Agent. Type 'exit' to quit.")
agent = Agent(system=prompt)
response=agent("what is the weight of toy poodles?")
print(response)

Welcome to the Agent. Type 'exit' to quit.
Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [27]:
response=average_dog_weight("Toy Poodle")
print(response)

a toy poodles average weight is 7 lbs


In [30]:
next_response="observation: {}".format(response)
agent(next_response)


"Answer: A Toy Poodle's average weight is 7 lbs."

In [31]:
agent.messages


[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'what is the weig

In [41]:
agent=Agent(system=prompt)

In [42]:
question="""I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""

In [43]:
agent(question)

'Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to find the combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [45]:
next_response = "observation: {}".format(average_dog_weight("Border Collie"))
print(next_response)

observation: a Border Collies average weight is 37 lbs


In [49]:
agent(next_response)
next_response = "observation: {}".format(average_dog_weight("Scottish Terrier"))
agent(next_response)

'Action: calculate: 37 + 20\nPAUSE'

In [51]:
next_response="observation :{}".format(calculate("37 + 20"))
agent(next_response)

'Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.'

In [52]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [53]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [57]:
question = """I have 2 dogs, a border collie and a cat. \
What is their multiple weight"""
query(question)

Thought: To find the multiple of the weights of the two dogs and the cat, I need to find the average weight of a Border Collie and then multiply it by the weight of the cat. However, I don't have the weight of the cat, so I will assume a typical cat weight for calculation purposes. I'll start by finding the average weight of a Border Collie.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: Now that I have the average weight of a Border Collie, which is 37 lbs, I need to assume a typical weight for a cat. A common average weight for a domestic cat is around 10 lbs. I will calculate the multiple of the weights of the Border Collie and the cat using these values.

Action: calculate: 37 * 10
PAUSE
 -- running calculate 37 * 10
Observation: 370
Answer: The multiple of the weight of a Border Collie and a typical cat is 370 lbs.


In [54]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to find the combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.
